In [1]:
import pandas as pd 
import numpy as np
df = pd.read_parquet('../scripts/df_merged.parquet', engine='pyarrow')

In [2]:
df.shape

(1831533, 10)

In [3]:
unique_ids = df['objectId'].unique().tolist()

In [4]:
condition = df['source'] == 0
df['dc_sigflux_weight'] = np.where(condition, 1 / (df['dc_sigflux'] ** 2), 1 / (df['dc_sigflux'] ** 2))
df['nr_sigflux_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

In [5]:
df[['source','dc_sigflux_weight']].head()

,source,dc_sigflux_weight
0,1,1.353949e+09
1,1,2.615609e+08
2,0,7.029790e+07
3,0,5.681616e+07
4,1,2.325079e+08


In [152]:
import time 
start_time = time.time()



m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?
#print(Q)

#T = {k: df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:100]}
d = {k: 0 for k in unique_ids[0:10]}
#print(d)

count = 0

for k in unique_ids[0:10]:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_sigflux_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn 
        for j in range(0,m): 
            
            alpha = 0
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                s_n = (f[j:]*h[j] * w[j:])
                s_d = (h[j]**2 * w[j:])

                alpha += s_n / s_d
                #print("alpha", alpha)
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                d[k] += ((f[j:] - alpha * h[j])**2) * w[j:]
               #print()
               #print(d)

            #break
            
    if (d[k] <= factor).all():
        #print(d[k],k)
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")
       

1
Elapsed time: 0.945817232131958 seconds


In [153]:
d

{'ZTF17aaaeclk': array([ 216.8653502 ,   27.75966475,   44.73559293,   27.75966475,
        1927.35239901,  665.04807819, 1098.22716313,  968.89136299,
         678.07470059,  739.9763153 , 1127.45699085,   27.75966475,
         786.95693348,  280.12362594,   25.02810584,  838.1556615 ,
        1197.17909242,   27.75966475,   45.92358266, 2455.06416008,
          27.75966475,   27.75966475,   27.75966475,   27.75966475,
          27.75966475,   27.75966475,   27.75966475,  486.9850509 ,
          27.75966475,   27.75966475, 2915.96309434]),
 'ZTF17aaarimh': array([4014.39606932,   58.28918116,  722.1199834 , 5882.56369885,
        3749.38799044,   58.28918116,  877.52326498, 6318.08368799,
        3999.45522757, 3339.22666933, 2677.36242672, 3824.20336644,
         367.81002386,  959.46320702, 3512.2363699 , 2565.7427527 ,
          58.28918116, 2670.41663497,   58.28918116,   58.28918116,
        2173.56178984,   58.28918116,   58.28918116,   58.28918116,
        1984.9297928 , 2869.6

In [151]:
import time 
start_time = time.time()



m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['dc_flux']].iloc[:2*m]#.values # j:j+m ?
#print(Q)

#T = {k: df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:100]}
d = {k: 0 for k in unique_ids[0:10]}
#print(d)

count = 0

for k in unique_ids[0:10]:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_sigflux_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn 
        for j in range(0,m): 
            f = T['dc_flux'].values[j:]
            w = T['dc_sigflux_weight'].values[j:]
            h = np.tile(Q, (len(f) // 2, 1)).ravel() # array of h for r and g successive of the vectorisation
            print(len(f), k)
            s_n = (f*h*w)
            s_d = (h**2 * w)
            
            s = s_n / s_d
            #print(s.shape)
            alpha = s[::2] + s[1::2]
            #print(alpha.shape)
            
        for j in range(0,m):
            f = T['dc_flux'].values[j:]
            w = T['dc_sigflux_weight'].values[j:]
            h = np.tile(Q, (len(f) // 2, 1)).ravel() # array of h for r and g successive of the vectorisation
            
            alp = np.repeat(alpha, 2)
            dd = ((f - alp * h)**2) * w
            d[k] = dd[::2] + dd[1::2]
            
            #print(dd)
            
    if (d[k] <= factor).all():
        #print(d[k],k)
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")
       

62 ZTF17aaaeclk
58 ZTF17aaarimh
30 ZTF18acevrat
62 ZTF18abhxxas
42 ZTF18aakgeyw
54 ZTF18acoyqvk
54 ZTF17aactufx
62 ZTF18abcoyzc
62 ZTF18abnyhra
62 ZTF18abmwqzf
0
Elapsed time: 0.9217839241027832 seconds


In [137]:
d

{'ZTF17aaaeclk': array([ 216.8653502 ,   27.75966475,   44.73559293,   27.75966475,
        1927.35239901,  665.04807819, 1098.22716313,  968.89136299,
         678.07470059,  739.9763153 , 1127.45699085,   27.75966475,
         786.95693348,  280.12362594,   25.02810584,  838.1556615 ,
        1197.17909242,   27.75966475,   45.92358266, 2455.06416008,
          27.75966475,   27.75966475,   27.75966475,   27.75966475,
          27.75966475,   27.75966475,   27.75966475,  486.9850509 ,
          27.75966475,   27.75966475, 2915.96309434]),
 'ZTF17aaarimh': array([4014.39606932,   58.28918116,  722.1199834 , 5882.56369885,
        3749.38799044,   58.28918116,  877.52326498, 6318.08368799,
        3999.45522757, 3339.22666933, 2677.36242672, 3824.20336644,
         367.81002386,  959.46320702, 3512.2363699 , 2565.7427527 ,
          58.28918116, 2670.41663497,   58.28918116,   58.28918116,
        2173.56178984,   58.28918116,   58.28918116,   58.28918116,
        1984.9297928 , 2869.6

In [129]:
alpha

array([2.        , 2.02773284, 2.0679294 , 2.02773284, 1.85734727,
       2.75171466, 1.82118907, 2.85939335, 1.75434098, 2.694254  ,
       1.72462479, 2.02773284, 1.37455808, 1.65021628, 1.95980345,
       1.75997982, 1.77748895, 2.02773284, 1.95980345, 2.51118088,
       2.02773284, 2.02773284, 2.02773284, 2.02773284, 2.02773284,
       2.02773284, 2.02773284, 2.33747332, 2.02773284, 2.02773284,
       2.6482928 ])

In [128]:
np.repeat(alpha, 2)

array([2.        , 2.        , 2.02773284, 2.02773284, 2.0679294 ,
       2.0679294 , 2.02773284, 2.02773284, 1.85734727, 1.85734727,
       2.75171466, 2.75171466, 1.82118907, 1.82118907, 2.85939335,
       2.85939335, 1.75434098, 1.75434098, 2.694254  , 2.694254  ,
       1.72462479, 1.72462479, 2.02773284, 2.02773284, 1.37455808,
       1.37455808, 1.65021628, 1.65021628, 1.95980345, 1.95980345,
       1.75997982, 1.75997982, 1.77748895, 1.77748895, 2.02773284,
       2.02773284, 1.95980345, 1.95980345, 2.51118088, 2.51118088,
       2.02773284, 2.02773284, 2.02773284, 2.02773284, 2.02773284,
       2.02773284, 2.02773284, 2.02773284, 2.02773284, 2.02773284,
       2.02773284, 2.02773284, 2.02773284, 2.02773284, 2.33747332,
       2.33747332, 2.02773284, 2.02773284, 2.02773284, 2.02773284,
       2.6482928 , 2.6482928 ])

In [97]:
Q.values

array([[0.0002863 ],
       [0.00063625]])

In [107]:
Q.values
Q = np.array([[0],
              [1]])
h = np.tile(Q, (len(f) // 2, 1)).ravel()

In [108]:
h*f

array([0.        , 0.00599112, 0.        , 0.00508321, 0.        ,
       0.0059388 , 0.        , 0.00571934, 0.        , 0.00470789,
       0.        , 0.00508321, 0.        , 0.00470789, 0.        ,
       0.00522087, 0.        , 0.00574328, 0.        , 0.00550307,
       0.        , 0.00508321, 0.        , 0.00470789, 0.        ,
       0.00470789, 0.        , 0.003192  , 0.        , 0.00470789,
       0.        , 0.00470789, 0.        , 0.00508321, 0.        ,
       0.00508321, 0.        , 0.00508321, 0.        , 0.00508321,
       0.        , 0.00470789, 0.        , 0.00508321, 0.        ,
       0.00508321, 0.        , 0.00508321, 0.        , 0.00508321,
       0.        , 0.00508321, 0.        , 0.00508321, 0.        ,
       0.00508321, 0.        , 0.00425378])

In [112]:
f = T['dc_flux'].values[1:]

f

array([0.00599112, 0.00144109, 0.00508321, 0.00144109, 0.0059388 ,
       0.00194996, 0.00571934, 0.00194996, 0.00470789, 0.00144109,
       0.00508321, 0.00144109, 0.00470789, 0.00194996, 0.00522087,
       0.0011429 , 0.00574328, 0.00110097, 0.00550307, 0.00111138,
       0.00508321, 0.00144971, 0.00470789, 0.00144109, 0.00470789,
       0.00144109, 0.003192  , 0.00154019, 0.00470789, 0.00108359,
       0.00470789, 0.00144109, 0.00508321, 0.00127979, 0.00508321,
       0.00144109, 0.00508321, 0.00144109, 0.00508321, 0.00110401,
       0.00470789, 0.00144109, 0.00508321, 0.00144109, 0.00508321,
       0.00144109, 0.00508321, 0.00119462, 0.00508321, 0.00171222,
       0.00508321, 0.00144109, 0.00508321, 0.00144109, 0.00508321,
       0.00113502, 0.00425378])

In [99]:
Q

,dc_flux
0,0.000286
1,0.000636


In [70]:
Q = np.array([[0],
              [1]])

T = np.array([2,2,2,2,2,2])
T*Q

array([[0, 0, 0, 0, 0, 0],
       [2, 2, 2, 2, 2, 2]])

In [106]:
np.tile(Q, (len(T) // 2, 1)).ravel()

array([0.0002863 , 0.00063625, 0.0002863 , 0.00063625, 0.0002863 ,
       0.00063625, 0.0002863 , 0.00063625, 0.0002863 , 0.00063625,
       0.0002863 , 0.00063625, 0.0002863 , 0.00063625, 0.0002863 ,
       0.00063625, 0.0002863 , 0.00063625, 0.0002863 , 0.00063625,
       0.0002863 , 0.00063625, 0.0002863 , 0.00063625, 0.0002863 ,
       0.00063625, 0.0002863 , 0.00063625, 0.0002863 , 0.00063625,
       0.0002863 , 0.00063625, 0.0002863 , 0.00063625, 0.0002863 ,
       0.00063625, 0.0002863 , 0.00063625, 0.0002863 , 0.00063625,
       0.0002863 , 0.00063625, 0.0002863 , 0.00063625, 0.0002863 ,
       0.00063625, 0.0002863 , 0.00063625, 0.0002863 , 0.00063625,
       0.0002863 , 0.00063625, 0.0002863 , 0.00063625, 0.0002863 ,
       0.00063625, 0.0002863 , 0.00063625])

In [104]:
s= T*np.tile(Q, (len(T) // 2, 1)).ravel()
s

ValueError: Unable to coerce to Series, length must be 3: given 58

In [77]:
s[::2] + s[1::2]

array([2, 2, 2])

In [150]:
df[df['objectId'] =='ZTF17aaadyyl' ]

,objectId,candid,jd,mjd,fid,dc_flux,dc_sigflux,nr_flux,nr_sigflux,source,dc_sigflux_weight,nr_sigflux_weight
1566,ZTF17aaadyyl,1356463635315010058,2.459111e+06,59082,2,0.000348,0.000014,0.000548,0.000008,1,5.454123e+09,1.533048e+10
1567,ZTF17aaadyyl,1356463635315010058,2.459111e+06,59083,2,0.000334,0.000061,0.000548,0.000000,0,2.710012e+08,0.000000e+00
1568,ZTF17aaadyyl,1356463635315010058,2.459111e+06,59084,2,0.000334,0.000061,0.000548,0.000000,0,2.710012e+08,0.000000e+00
1569,ZTF17aaadyyl,1356463635315010058,2.459111e+06,59085,2,0.000324,0.000018,0.000548,0.000008,1,3.080234e+09,1.533048e+10
1570,ZTF17aaadyyl,1356463635315010058,2.459111e+06,59086,2,0.000337,0.000019,0.000548,0.000008,1,2.719534e+09,1.533048e+10
1571,ZTF17aaadyyl,1356463635315010058,2.459111e+06,59087,2,0.000334,0.000061,0.000548,0.000000,0,2.710012e+08,0.000000e+00
1572,ZTF17aaadyyl,1356463635315010058,2.459111e+06,59088,2,0.000334,0.000061,0.000548,0.000000,0,2.710012e+08,0.000000e+00
1573,ZTF17aaadyyl,1356463635315010058,2.459111e+06,59089,2,0.000334,0.000061,0.000548,0.000000,0,2.710012e+08,0.000000e+00
1574,ZTF17aaadyyl,1356463635315010058,2.459111e+06,59090,2,0.000340,0.000013,0.000548,0.000008,1,6.281327e+09,1.533048e+10
1575,ZTF17aaadyyl,1356463635315010058,2.459111e+06,59091,2,0.000332,0.000015,0.000548,0.000008,1,4.661313e+09,1.533048e+10


In [ ]:

import timeit

def approach1():
    condition = df['source'] == 0
    df['dc_sigflux_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
    df['nr_sigflux_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

def approach2():
    df['dc_sigflux_weight'] = 1 / (df['dc_sigflux'] ** 2)
    df['nr_sigflux_weight'] = 1 / (df['nr_sigflux'] ** 2)
    df['dc_sigflux_weight'] = df['dc_sigflux_weight'] * (df['source'] != 0)
    df['nr_sigflux_weight'] = df['nr_sigflux_weight'] * (df['source'] != 0)

time_approach2 = timeit.timeit(approach2, number=10)
time_approach1 = timeit.timeit(approach1, number=10)

print("1 execution time:", time_approach1)
print("2 execution time:", time_approach2)


In [79]:
m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?

T = {unique_ids.index(k): df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:1]}
d = {unique_ids.index(k): np.zeros(int(len(T[k])/2), dtype=np.float64) for k in unique_ids[0:1]}

for j in range(1,m+1): 
    #h = Q.values     
    
    
            for filt in np.unique(T[k]['fid']):
                mask = T[k]['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[k][mask]['dc_flux'].values
                w = T[k][mask]['dc_sigflux_weight'].values

                s_n = (f[-1+j:]*h[j-1] * w[-1+j:])
                s_d = (h[j-1]**2 * w[-1+j:])

                alpha = s_n / s_d
                #print("alpha", alpha)
                
                d[k] += ((f[-1+j:] - alpha * h[j-1])**2) * w[-1+j:]

                #break
            
if (d[k] <= factor).any():
        #print(d)
        count +=1
        #print(count, k)
        
print(count)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices